In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shap
from lime.lime_tabular import LimeTabularExplainer


# 1: Transform data to dense metrics

X_train_transformed = xgb_pipeline.named_steps['preprocessor'].transform(X_train)
X_train_dense = X_train_transformed.toarray() if hasattr(X_train_transformed, "toarray") else X_train_transformed

X_test_transformed = xgb_pipeline.named_steps['preprocessor'].transform(X_test)
X_test_dense = X_test_transformed.toarray() if hasattr(X_test_transformed, "toarray") else X_test_transformed


# 2: Features

num_features = numeric_features
cat_features = list(xgb_pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features))
txt_transformer = xgb_pipeline.named_steps['preprocessor'].named_transformers_['txt']
tfidf_feature_names = txt_transformer.get_feature_names_out()

feature_names = np.concatenate([num_features, cat_features, tfidf_feature_names])


# 3: SHAP Explainer

explainer = shap.TreeExplainer(xgb_pipeline.named_steps['classifier'])
shap_values = explainer.shap_values(X_test_dense)


# 4: SHAP summary plot (global)

shap.summary_plot(shap_values, X_test_dense, feature_names=feature_names)


# 5: SHAP force plot (local)

i = 0  # index of sample to explain
shap.initjs()

# Create a white-background figure for force plot
fig, ax = plt.subplots(figsize=(12,3))
plt.axis('off')
force_plot = shap.force_plot(
    explainer.expected_value, 
    shap_values[i], 
    X_test_dense[i], 
    feature_names=feature_names, 
    matplotlib=True
)
plt.show()


# 6: LIME Explainer (local)

lime_explainer = LimeTabularExplainer(
    training_data=np.array(X_train_dense),
    feature_names=feature_names,
    class_names=['Retained', 'Churned'],
    mode='classification'
)

exp = lime_explainer.explain_instance(
    data_row=X_test_dense[i],
    predict_fn=xgb_pipeline.named_steps['classifier'].predict_proba
)

# Show explanation in notebook
exp.show_in_notebook(show_table=True)

# save as HTML
exp.save_to_file('lime_explanation_sample0.html')
